In [36]:
%config InlineBackend.figure_formats = ['svg']
%cp -v ../coronavirus.py .
from coronavirus import *

../coronavirus.py -> ./coronavirus.py


In [37]:
d, c = fetch_deaths(), fetch_cases()

countries = d.index
countries2 = c.index
assert (countries2 == countries).all()

In [48]:
def modify_template(templatefile, output_file_name, mappings, wwwroot):
    """Create concrete *.ipynb file from template
    - templatefile: the template with placeholders to be substituted
    - mappings: dictiorany with placeholders as keys, and values to be substituted
    - output_file_name: name to write modified file to
    - wwwroot: directory in which the output file should be written
    """
    # open template
    with open(templatefile, "tr") as f_template:
        template = f_template.read()
    for key in mappings:
        template = template.replace(key, mappings[key])
    with open(os.path.join(wwwroot, output_file_name), "tw") as f:
        f.write(template)
    print(f"Written file to {output_file_name}")


In [83]:
def check_country_name_is_known(name):
    d = fetch_deaths()
    assert name in d.index, f"{name} is unknown. Known countries are {sorted(d.index)}"

def sanitize(name):
    """Given a country name as a string, sanitise it for use as URL and filename: 
    - get rid of spaces, commas
    
    return cleaned string.
    
    (Leave umlaute for now)
    """
    s = name.replace(" ", "-")
    s = s.replace(",", "-")
    return s
    
    
def get_binder_url(notebook):
    base = "https://mybinder.org/v2/gh/https%3A%2F%2Ffangohr.github.io%2Fcoronavirus/master?filepath="
    return base + notebook


def create_ipynb_for_country(country, wwwroot):
    check_country_name_is_known(country)
    
    output_file_name =  f"{country}.ipynb"
    output_file_path = os.path.join(wwwroot, output_file_name)
    
    # country = sanitize(country)
    mappings = {
        "%title%" : country,
        "%title2%" : "(data from Johns Hopkins)",
        "%country%" : country,
        "%binderurl%" : get_binder_url(output_file_path)
    }

    modify_template("template-country.ipynb", output_file_name, mappings, wwwroot)
    assert os.path.exists(output_file_path)
    return output_file_name


In [ ]:
import subprocess
def nbconvert_ipynb2html(ipynb_path, wwwroot):
    
    # copy file to run the notebook
    command = f"cp -fv ../coronavirus.py {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # copy requirements (needed for binder)
    command = f"cp -fv ../requirements.txt {wwwroot}"
    subprocess.check_call(command, shell=True)
    
    # execute notebook and create html copy from it
    command = f"""jupyter-nbconvert "{os.path.join(wwwroot, ipynb_path)}" """ + \
                " --to html --execute"
    print(f"Command = {command}")
    output = subprocess.check_call(command, shell=True)

    # compute output path
    output_file_name = os.path.splitext(ipynb_path)[0] + ".html"
    assert os.path.exists(os.path.join(wwwroot, output_file_name))
    
    return output_file_name
    

In [85]:
def create_md_index_list(title, links):
    lines = []
    lines.append(title)
    for name, (path_html, path_ipynb) in links.items():
        lines.append(f"* [{name}]({path_html})")
    return "\n".join(lines)
        

In [86]:
@joblib_memory.cache
def create_html_for_john_hopkins_countries(wwwroot):
    if not os.path.exists(wwwroot):
        print("To put the html into github repo for webhosting, run")
        print('"git clone git@github.com:fangohr/coronavirus.git wwwroot" or similar')
        os.mkdir(wwwroot)
    d, c = fetch_deaths(), fetch_cases()

    countries = d.index
    countries2 = c.index
    assert (countries2 == countries).all()

    created_files = {}
    country = "Italy"
    #for country in ["Italy", "Germany", "US"]:
    for i, country in enumerate(countries):
        print(f"Processing {i+1}/{len(countries)}")
        ipynb_path = create_ipynb_for_country(country, wwwroot=wwwroot)
        html_path = nbconvert_ipynb2html(os.path.join(ipynb_path), wwwroot=wwwroot)
        created_files[country] = html_path, ipynb_path
    return created_files

In [87]:
def create_index_page(sections, rootname, wwwroot):
    """Sections is dictionary: key is title, value is markdown text"""
    md_file = rootname + ".md"
    with open(os.path.join(wwwroot, md_file), "tw") as f:
        for section in sections:
            f.write(f"# {section}\n\n")
            f.write(sections[section])
    print(f"Written overview to {md_file}.")
    html_file = rootname + ".html"
    subprocess.check_call(f"pandoc -t html -o {html_file} {md_file}", shell=True)
    return html_file

In [ ]:
wwwroot = "wwwroot"
created_files = create_html_for_john_hopkins_countries(wwwroot)
index_md = create_md_index_list("Countries (data from Johns Hopkins University)", 
                                created_files)
create_index_page(sections={"Links to static pages for each country" : index_md}, 
                  rootname="index", 
                  wwwroot=wwwroot)


Written file to Afghanistan.ipynb
Command = jupyter-nbconvert "wwwroot/Afghanistan.ipynb"  --to html --execute
Output is 
Written file to Albania.ipynb
Command = jupyter-nbconvert "wwwroot/Albania.ipynb"  --to html --execute
Output is 
Written file to Algeria.ipynb
Command = jupyter-nbconvert "wwwroot/Algeria.ipynb"  --to html --execute
Output is 
Written file to Andorra.ipynb
Command = jupyter-nbconvert "wwwroot/Andorra.ipynb"  --to html --execute
Output is 
Written file to Angola.ipynb
Command = jupyter-nbconvert "wwwroot/Angola.ipynb"  --to html --execute
Output is 
Written file to Antigua and Barbuda.ipynb
Command = jupyter-nbconvert "wwwroot/Antigua and Barbuda.ipynb"  --to html --execute
Output is 
Written file to Argentina.ipynb
Command = jupyter-nbconvert "wwwroot/Argentina.ipynb"  --to html --execute
Output is 
Written file to Armenia.ipynb
Command = jupyter-nbconvert "wwwroot/Armenia.ipynb"  --to html --execute
Output is 
Written file to Australia.ipynb
Command = jupyter-nbco

In [63]:
print(create_md_index_list("Countries (data from Johns Hopkins University)", created_files))

Countries (data from Johns Hopkins University)
* [Antigua and Barbuda](Antigua and Barbuda.html)
* [Argentina](Argentina.html)
* [Armenia](Armenia.html)
* [Australia](Australia.html)
